<a href="https://colab.research.google.com/github/acanetti/Profiler/blob/master/Democratie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
! pip install wiktionaryparser # Installation parser 

In [22]:
import pandas as pd 
import numpy as np 
import os  
PATH ='drive/My Drive/data' 
from time import sleep
from IPython.display import clear_output, display 
import multiprocessing
import threading 
import wiktionaryparser as Wikpars
from multiprocessing import Process, Value, Array
import builtins
print(' core available ' , multiprocessing.cpu_count() ) 

 core available  4


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import données Démocratie 

In [24]:
df=pd.read_csv(PATH + '/democratie_clean.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
df.head(2)

,Unnamed: 0,title,authorZipCode,Q_0,Q_1,Q_2,Q_3,Q_4,Q_5,Q_6,Q_7,Q_8,Q_9,Q_10,Q_11,Q_12,Q_13,Q_14,Q_15,Q_16,Q_17,Q_18,Q_19,Q_20,Q_21,Q_22,Q_23,Q_24,Q_25,Q_26,Q_27,Q_28,Q_29,Q_30,Q_31,Q_32,Q_33,Q_34,Q_35,Q_36
0,0,Les augmentations de rémunérations,79190,Le citoyen,Non,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,Afin d’éviter de creuser les inégalités ne plu...
1,1,rénover l'enquête publique pour en faire un vr...,1800,Un instrument de démocratie locale à modernise...,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//,//


## Concatenation de toute les colonnes pour créer dictionnaire 

In [0]:
concat_all='' 
for i in range(37): 
    concat_all += df[f'Q_{i}'] + ' % '

In [0]:
concat_np = concat_all.values 

# Word Vectorizer 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer=TfidfVectorizer(min_df=0.0001, max_features=90000)


In [0]:
X = vectorizer.fit_transform(concat_np)

In [31]:
print( f' nombre de mots dans dictionnaire {len(vectorizer.get_feature_names())}')

 nombre de mots dans dictionnaire 37058


In [0]:
List_dic = np.array(vectorizer.get_feature_names()) # Création de l'array des elements 

In [0]:
Alpha_dex= [List_dic[i].isalpha() for i in range(len(List_dic))] # Récupération des elements alphabétique seulement 
alpha_voc = List_dic[Alpha_dex]

In [34]:
print(f'nombre de mots alphabetique dans le dictionnaire :{len(alpha_voc)}')

nombre de mots alphabetique dans le dictionnaire :36439


In [0]:
size = len(alpha_voc)
def type_finder( x : str  ) -> str:
    
    tmp='NULL'
    parser = Wikpars.WiktionaryParser()  
    parser.set_default_language('french')
    word = parser.fetch(x)  
    try : 
        tmp= word[0]['definitions'][0]['partOfSpeech'] 
    except : pass  
    return tmp 


In [0]:

size = len(alpha_voc)
strArr = np.empty(size, dtype='<U2186')
 
def testy(start,stop): 
    
    for val in range(start,stop) :    
       
        Dict[alpha_voc[val]] = type_finder(alpha_voc[val])
        proc_name = multiprocessing.current_process().name 
        clear_output(wait=False)


In [0]:
def Create_process(core : int = multiprocessing.cpu_count() ) -> list : 
    manager = multiprocessing.Manager()
    global Dict 
    Dict = manager.dict() 
    
    process_list=[]
    size = len(alpha_voc) // core 
    step = core 
    for i in range(core): 
        proc=multiprocessing.Process(target=testy, args=(int(size*i),int(size*(i+1)))) 
        process_list.append(proc)  
    
    return process_list




In [0]:
Proc = Create_process()

In [0]:
for pro in Proc: 
    pro.start()

In [80]:


for pro in Proc : 
    print (pro.name , pro.is_alive()) 
alprint('Size of the Dic;' ,len(Dict)) 

Process-7 False
Process-8 False
Process-9 False
Process-10 False
Size of the Dic; 36436


In [0]:
alphadic={}
for key,val in Dict.items(): 
    alphadic[key]=val

In [90]:
jbl.dump(alphadic,'alphadic.pkl')

['alphadic.pkl']

71899